# Sistema de tipos

Dos de los ingredientes más importantes que caracterizan a Julia y permiten que sea un lenguaje rápido son la **inferencia de tipos** y la **especialización de funciones con respecto al tipo**. En conjunto, estos ingredientes permiten el *despacho múltiple*. A primera vista, muchos ven el despacho múltiple solo como algo lindo que viene con el lenguaje, pero, en realidad, se trata de un aspecto esencial con raíces profundas en la base del lenguaje que permiten escribir código rápido.

En este contexto, el andamiaje de Julia es su sistema de tipos. Al nivel de la CPU, todos los objetos tienen un tipo. El tipo de un objeto permite interpretar la memoria y saber qué hacer en un llamado a una función. Si no se conoce el tipo de los objetos, no se puede computar nada, ya que la CPU necesita saber cómo interpretar los bits.  Algunos lenguajes son más explícitos acerca de los tipos, y otros tratan de esconder el tipo del usuario. 

En Julia, **todos los objetos tienen un tipo**:

In [2]:
typeof(1.0)

Float64

In [3]:
typeof(1)

Int64

In [4]:
typeof("Hola")

String

In [5]:
typeof(:a)

Symbol

Incluso los tipos son objetos y, por lo tanto, tienen tipo

In [6]:
typeof(Float64)

DataType

### Relación de subtipo-supertipo

Algunos tipos son subtipo de otros tipos. La relación subitpo-supertipo se designa con el símbolo `<:`

Por ejemplo:

In [9]:
Int <: Real

true

In [8]:
Real <: Number

true

Todos los tipos son subtipo del tipo `Any`

In [10]:
Int <: Any

true

In [11]:
String <: Any

true

In [12]:
Vector{Float64} <: Any

true

La relación `<:` satisface:

1. Reflexividad: `A<:A` para todo tipo `A`
2. Antisimetría: si `A<:B` y `B<:A`, entonces `A==B` para todo tipo `A` y `B`
3. Transitividad: si `A<:B` y `B<:C`, entonces `A<:C` para todo tipo `A`, `B` y `C`

Por lo tanto, `<:` es un orden parcial sobre los tipos, con `Any` como máximo. 

Los tipos puede visualizarse en un diagrama de Hasse. El siguiente es el diagrama de los tipos numéricos en la base de Julia:

<div align="center">
  <img src="./img/tipos.png" alt="Tipos numéricos en Julia" width="850"/>
</div>

Crédito de la imagen: [The Julia Express](https://github.com/bkamins/The-Julia-Express)

### Tipos abstractos y concretos

En Julia, los tipos se clasifican en **abstractos** y **concretos**. Los **tipos abstractos** son solo nodos en el diagrama que sirven para organizar los tipos en una jerarquía. No son isntanciables. Es decir, no se puede construir objetos cuyo tipo sea abstracto.

Los tipos abstractos se declaran con `abstract type`. Por ejemplo, así sería la implementación de algunos de los tipos numéricos en Julia:

```julia
abstract type Number end

abstract type Real <: Number end

abstract type AbstractFloat <: Real end
abstract type Integer <: Real end

abstract type Signed <: Integer end
abstract type Unsigned <: Integer end
```

También podemos crear nuevos tipos abstractos:

In [14]:
abstract type Bebida end

abstract type Cafe <: Bebida end
abstract type Te <: Bebida end
abstract type Vino <: Bebida end

Por otro lado, los **tipos concretos** son los que pueden instanciarse y **no admiten subtipos**. 

Los tipos concretos permiten organizar datos. Pueden definirse tipos compuestos con `struct`:

In [15]:
struct Expreso <: Cafe
    marca::String
    precio::Float64
end

Se instancian de la siguiente manera:

In [16]:
Expreso("Cabrales", 1300.0)

Expreso("Cabrales", 1300.0)

Y, como dijimos, no admiten subtipos. Si tratamos de definir un tipo como subtipo de uno concreto, Julia arrojará un error:

In [17]:
struct Ristretto <: Expreso
    marca::String
    precio::Float64
end

ErrorException: invalid subtyping in definition of Ristretto: can only subtype abstract types.

Cuando se definen con `struct`, los tipos son inmutables:

In [18]:
a = Expreso("Cabrales", 1300)

a.precio = 1500

ErrorException: setfield!: immutable struct of type Expreso cannot be changed

Usando `mutable struct` en lugar de `struct` podemos definir tipos concretos mutables

In [19]:
mutable struct Americano <: Cafe
    marca::String
    precio::Float64
end

a = Americano("Cabrales", 1300)

Americano("Cabrales", 1300.0)

In [20]:
a.precio = 1500
a

Americano("Cabrales", 1500.0)

Por último, también podemos definir tipos concretos sin campos. Por ejemplo:

In [21]:
struct Agua <: Bebida end

Esta clase de tipos se denominan *singletons*, ya que solo admiten una única instancia.

In [22]:
agua1 = Agua()
agua2 = Agua()

agua1 === agua2

true

donde `===` es el operador de identidad, que verifica si dos variables refieren al mismo objeto (no si son iguales, sino idénticas). Por ejemplo, el famoso `nothing` de Julia es la instancia *singleton* del tipo `Nothing`.